In [ ]:
!pip install statsbombpy

In [ ]:
# Download the data ----

from statsbombpy import sb

# Download event data from the match that clinched the Premier League title for Leicester City.
# It was a 2-2 draw between Tottenham and Chelsea that eliminated Tottenham from catching up.
events = sb.events(match_id=3754092)

In [ ]:
# Start by classifying the start state and end state of each event ----

import numpy as np
import pandas as pd

data = (
    events
        .sort_values('index')
        .assign(
            # Extract x and y locations as numerics
            # It's a little bit tricky because some of the entries are NaN, not list
            loc_x=lambda x: x['location'].apply(lambda y: y[0] if isinstance(y, list) else np.nan),
            loc_y=lambda x: x['location'].apply(lambda y: y[1] if isinstance(y, list) else np.nan),
            # Unlike Rudd's work, let's split the field into 30 zones (6 for x, 5 for y)
            zone_x=lambda x: np.select(
                condlist=[x['loc_x'] < 20, x['loc_x'] < 40, x['loc_x'] < 60, x['loc_x'] < 80, x['loc_x'] < 100, x['loc_x'] < 120],
                choicelist=['0', '1', '2', '3', '4', '5'],
                default=None
            ),
            zone_y=lambda x: np.select(
                condlist=[x['loc_y'] < 16, x['loc_y'] < 32, x['loc_y'] < 48, x['loc_y'] < 64, x['loc_y'] < 80],
                choicelist=['0', '1', '2', '3', '4'],
                default=None
            ),
            zone=lambda x: x['zone_x'] + x['zone_y']
        )
        # According to Rudd's Markov model, we only consider these three event types
        .loc[lambda x: x['type'].isin(['Pass', 'Carry', 'Shot'])]
        .assign(
            # The starting state is the zone on the field where the event started, unless it's
            # one of the special game states (Free Kick, Corner, Throw-in). We skip Penalty because
            # there were no penalties in this game, and for simplicity we do not distinguish
            # between short and long like Rudd did.
            start_state=lambda x: np.select(
                condlist=[x['pass_type'] == 'Free Kick', x['pass_type'] == 'Corner', x['pass_type'] == 'Throw-in'],
                choicelist=['Free Kick', 'Corner', 'Throw-in'],
                default=x['zone']
            ),
            # We define the ending state to be the starting state of the next event. We make
            # exceptions for terminal states, which happen on a goal or a change of possession.
            end_state=lambda x: np.select(
                condlist=[x['shot_outcome'] == 'Goal', ~(x['possession_team_id'] == x['possession_team_id'].shift(-1))],
                choicelist=['Goal', 'End'],
                default=x['start_state'].shift(-1)
            )
        )
)


In [ ]:
# Count the number of transitions observed between each start state and each end state ----

transition_counts_raw = data.value_counts(['start_state', 'end_state'])
# Supplement our counts with fake transitions from each terminal state to itself.
# This way the terminal states are represented among the start states, which is
# necessary for the matrix computation below.
transition_counts_raw.loc['End', 'End'] = 1
transition_counts_raw.loc['Goal', 'Goal'] = 1

# Pivot the dataframe of transition counts into a matrix structure
# (one row for each start state, one column for each end state).
transition_counts = (
    transition_counts_raw
        .reset_index()
        .pivot_table(index='start_state', columns='end_state', values=0)
        .fillna(0)  # NaNs happen when the start state / end state pair is never observed (i.e. 0)
)

In [ ]:
# Divide each row by its sum to convert counts into probabilities ----

transition_probs = (
    transition_counts.div(transition_counts.sum(axis=1), axis=0)
)

In [ ]:
# Calculate the probability of ending up in each terminal state from each start state ----

# Set up an object to hold the result of the multiplication
terminal_transition_probs = transition_probs

# Multiply the matrix by itself 100 times
# (Rudd did 20, but we require more, probably because we allow transitions from a state to itself)
for i in range(100):
    terminal_transition_probs = terminal_transition_probs.dot(transition_probs)

# Extract the probability of goal being the terminal state from each start state
goal_prob = (
    terminal_transition_probs
    .reset_index()
    [['start_state', 'Goal']]
)

In [ ]:
# Produce player values ----

(
    data
        # Get the goal prob for the state before each action
        .merge(goal_prob, on='start_state', how='left')
        .rename(columns={'Goal': 'goal_prob_before'})
        # Get the goal prob for the state after each action
        .merge(goal_prob, left_on='end_state', right_on='start_state', how='left')
        .rename(columns={'Goal': 'goal_prob_after'})
        # Calculate the change in goal prob for each action
        .assign(goal_prob_diff=lambda x: x['goal_prob_after'] - x['goal_prob_before'])
        # Sum up the changes in goal prob for each player
        .groupby(['player_id', 'player'])
        ['goal_prob_diff']
        .sum()
        .sort_values(ascending=False)
)